In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from lxml import html
from time import sleep
import re
import warnings
warnings.simplefilter('ignore')
import requests_cache
import datetime

expire_after = datetime.timedelta(weeks=1)
requests_cache.install_cache(cache_name='nokisaki', backend='sqlite', expire_after=expire_after)

df = pd.read_csv('start.csv')

def parce_item(x):    
    if type(x) == str:
        x = re.sub(r"\s","",x)
        return x
    elif type(x) == list:
        lists = []
        for li in x:
            lists.append(re.sub(r"\s","",li)) 
        return lists
    else:
        print(type(x))

for i in range(1,10):
    sleep(1)
    print(i)
    request_url = f'https://business.nokisaki.com/spaces/detail/{2}'

    res = requests.get(request_url)

    source_block = BeautifulSoup(res.text, 'html.parser')
    # source_block

    source = html.fromstring(res.text)
    print(source.xpath("//h1/text()")[0])
    if source.xpath("//h1/text()")[0] != 'エラー':
        use_case_lists = ['販売可','車出店可','PR可','撮影可','教室/セミナー可','防音','鏡あり']
        facility_lists = ['駐車場','鍵','水道','扉','シャッター','床(砂利・土)','床(砂利・土以外)','電気コンセント','照明','暖房','冷房','屋根','トイレ','インターネット']

        name = source.xpath("//h1[@class='space-title']")[0].text

        price = source.xpath("//table//td/span[@class='price-amount']")[0].text

        space_info = source.xpath("//tr[@class='type-extent']/td/text()") # type/広さ

        space_type = space_info[0].split('/')[0]
        space_breadth = space_info[0].split('/')[1]

        use_cases = source.xpath("//div[contains(@class,'space-condition')]//span[contains(@class,'disactive')]/text()[2]")

        facility = source.xpath("//div[contains(@class,'space-setsubi')]//div[contains(@class,'expandable-content')]//div[contains(@class,'disactive')]/text()[2]")   

        address = ''.join(source.xpath("//div[contains(@class,'space-address')]/h3/text()"))

        discription = source.xpath("//div[@class='space-pr']/text()")

        row_data = {
            'name': parce_item(name), 
            'price': parce_item(price), 
            'address': parce_item(address), 
            'discription': parce_item(discription), 
            'space_type': parce_item(space_info[0].split('/')[0]),
            'space_breadth': parce_item(space_info[0].split('/')[1]),
            'use_case': list(set(parce_item(use_cases)) ^ set(use_case_lists)),
            'facility': list(set(parce_item(facility)) ^ set(facility_lists))
        }
        print(row_data)
        df = df.append(row_data, ignore_index=True, sort=False)
    else:
        row_data = {
            'name': '-',
            'price': '-',
            'address': '-',
            'discription': '-',
            'space_type': '-',
            'space_breadth': '-',
            'use_case': '-',
            'facility': '-',
        }
        print(row_data)
        df = df.append(row_data, ignore_index=True, sort=False)

df.to_csv("nokisaki.csv",index=False)



1
四ツ谷 絵本塾ホール
{'name': '四ツ谷絵本塾ホール', 'price': '27,500', 'address': '四ツ谷絵本塾ホール〒1600011東京都新宿区若葉1-22-16', 'discription': ['四ツ谷にある絵本塾ホールです。閑静な住宅街にあるホールは、コンサート、発表会、セミナー、など様々なイベント・催事にお使いいただくことができます。ご利用には、基本室内灯(蛍光灯)、机5台、イス60客、が含まれます。舞台照明、音響設備が別途料金にて利用可能です。45日前からのご予約に限り、特別料金にて受付けをしております。'], 'space_type': '屋内', 'space_breadth': '72.9㎡', 'use_case': ['教室/セミナー可'], 'facility': ['床(砂利・土以外)', '暖房', '冷房', '屋根', 'トイレ', '電気コンセント']}
2
四ツ谷 絵本塾ホール
{'name': '四ツ谷絵本塾ホール', 'price': '27,500', 'address': '四ツ谷絵本塾ホール〒1600011東京都新宿区若葉1-22-16', 'discription': ['四ツ谷にある絵本塾ホールです。閑静な住宅街にあるホールは、コンサート、発表会、セミナー、など様々なイベント・催事にお使いいただくことができます。ご利用には、基本室内灯(蛍光灯)、机5台、イス60客、が含まれます。舞台照明、音響設備が別途料金にて利用可能です。45日前からのご予約に限り、特別料金にて受付けをしております。'], 'space_type': '屋内', 'space_breadth': '72.9㎡', 'use_case': ['教室/セミナー可'], 'facility': ['床(砂利・土以外)', '暖房', '冷房', '屋根', 'トイレ', '電気コンセント']}
3
四ツ谷 絵本塾ホール
{'name': '四ツ谷絵本塾ホール', 'price': '27,500', 'address': '四ツ谷絵本塾ホール〒1600011東京都新宿区若葉1-22-16', 'discription': ['四ツ谷にある絵本塾ホールです。閑静な住宅街にあるホールは、コンサート、発表会、セミナー、など様々

In [19]:
url = r'https?://([\w-]+\.)+[\w-]+(/[\w-./?%&=]*)?'

In [20]:
source.xpath("//div[@class='image']/@style")[0]

'background-image: url(https://business.cdn.nokisaki.com/subs/0/2/S_54aa41b82c5f8.jpg);'

In [25]:
re.findall(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+',source.xpath("//div[@class='image']/@style")[0])[0]

'https://business.cdn.nokisaki.com/subs/0/2/S_54aa41b82c5f8.jpg)'